# markdown.obisidian.personal.reference
> Functions for managing references in an Obsidian.md math vault

In [ ]:
#| default_exp markdown.obsidian.personal.reference

In [ ]:
#| export
import glob
import itertools
import json
import os
from os import PathLike
from pathlib import Path
import re
import shutil
from typing import Union, Optional
import warnings


from trouver.helper.files_and_folders import path_name_no_ext
from trouver.helper.alphabet import alphabet_to_alphabet_group
from trouver.markdown.markdown.file import (
    MarkdownFile, MarkdownLineEnum
)
from trouver.markdown.markdown.heading import (
    heading_title
)
from trouver.markdown.obsidian.links import (
    ObsidianLink, LinkType, links_from_text
)
from trouver.markdown.obsidian.personal.authors import find_author_file
from trouver.markdown.obsidian.personal.index_notes import ( 
    convert_title_to_folder_name
)
from trouver.markdown.obsidian.personal.notes import (
    notes_linked_in_note
)
from trouver.markdown.obsidian.personal.note_type import (
    type_of_note, PersonalNoteTypeEnum
)
from trouver.markdown.obsidian.vault import(
    VaultNote, all_note_paths_by_name, note_path_by_name,
    NoteDoesNotExistError, NoteNotUniqueError, NotePathIsNotIdentifiedError
)

In [ ]:
import shutil
import tempfile
from unittest import mock

from fastcore.test import *

from trouver.helper.tests import _test_directory

# Getting a reference folder

In [ ]:
#| export
def index_note_for_reference(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, Path] # - The reference. Is either - a str, in which case the reference folder will be the folder containing the (unique) note of the name `_index_{reference}.md`, - or a `Path` object (not just a pathlike!) relative to `vault`, in which case the path will be the path to the reference folder. 
        ) -> VaultNote:
    """
    Returns the index note of the specified reference in the vault.
    
    Assumes that the reference folder has an index note named
    `_index_{reference_name}.md` and this note is the unique note in the vault
    with this filename.
    
    **Raises**

    - TypeError
        - If `reference` is not a str or PathLike.
    - NoteDoesNotExistError
        - If a note of the name `_index_{reference_name}.md` does not exist
        in the vault.
    """
    if (not isinstance(reference, str)
            and not isinstance(reference, PathLike)):
        raise TypeError(
            "Exspected `reference` to be a str or a PathLike, but got"
            f" {type(reference)} instead.")
    if isinstance(reference, str):
        reference_name = reference
        index_note = VaultNote(
            vault, name=f'_index_{reference_name}',
            update_cache=False)
    elif isinstance(reference, PathLike):
        reference_name = Path(reference).name
        index_note = VaultNote(
            vault, rel_path=Path(reference) / f'_index_{reference_name}.md')
    return index_note



The `index_note_for_reference` method obtains the index note of the specified reference in the vault.

In [ ]:
vault = _test_directory() / 'test_vault_5'
index_note_1 = index_note_for_reference(vault, reference='number_theory_reference_1')
assert index_note_1.name == '_index_number_theory_reference_1'

index_note_2 = index_note_for_reference(vault, reference=Path('number_theory') / 'number_theory_reference_1')
assert index_note_2.name == '_index_number_theory_reference_1'

In [ ]:
#| export
def reference_directory(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, Path], # - The reference. Is either - a str, in which case the reference folder will be the folder containing the (unique) note of the name `_index_{reference}.md`, - or a `Path` object (not just a pathlike!) relative to `vault`, in which case the path will be the path to the reference folder. 

        ) -> Path: # Relative to `vault`.
    """
    Returns the path to the reference directory in a vault.
    
    Assumes that the reference folder has an index note named
    `_index_{reference_name}.md`, this note is the unique note in the vault
    with this filename, and the cache in the `VaultNote` class for `vault` is
    updated.

    **Raises**

    - TypeError
        - If `reference` is not a str or PathLike.
    - 
    
    """
    index_note = index_note_for_reference(vault, reference)
    if index_note.exists(update_cache=True):
        return Path(index_note.path(relative=True)).parent
    else:
        raise NoteDoesNotExistError.from_note_name(index_note.name)

The `reference_directory` method obtains the root directory for the reference:

In [ ]:
vault = _test_directory() / 'test_vault_5'
dir = reference_directory(vault, reference='number_theory_reference_1')

If the vault does not have an index note for the specified reference, then a `NoteDoesNotExistError` is raised:

In [ ]:
with ExceptionExpected(NoteDoesNotExistError):
    thing = reference_directory(vault, reference='bad_number_theory_reference_without_an_index_note')
    print(thing)

## Deleting a reference folder

In [ ]:
#| export
def delete_reference_folder(
        vault: PathLike, # The vault in which the reference folder resides.
        reference: Union[str, PathLike], # The reference to delete. Is either a str, in which case the folder to delete will be the folder containing the (unique) note of the name `_index_{reference}.md`, or a path relative to `vault`. 
        verbose: bool = True,
        confirm: bool = True # If `True`, prompts the user to confirm the deletion of the folder.
        ) -> None:
    """
    Deletes a reference folder along with the associated template note
    and the reference note, both of which are outside the reference folder.
    
    Assumes that
    - the reference folder, if it exists, has an index note named
    `_index_{reference_name}.md` and this note is the unique note in the vault
    with this filename.
    - the template note, if it exists, is named `_template_{reference_name}.md`
    and is the unique note in the vault with this filename.
    - the reference note, if it exists, is named `_reference_{reference_name}.md`
    and is the unique note in the vault with this filename.
    
    If the template/reference note for the reference is not unique, then the
    deletion does not proceed. On the other hand, even if a template/reference
    note does not exist, then the deletion proceeds.

    Note that links to notes in the reference folder are preserved.
    
    **Raises**

    - FileNotFoundError
        - If the specified reference folder does not exist.
    - NoteDoesNotExistError
        - If the index note for the reference folder does not exist in the
        vault.
    - NoteNotUniqueError
        - If the index note, template note, or reference note for the reference
        folder is not unique in the vault.
    """
    try: 
        reference_path = reference_directory(vault, reference)
        absolute_path = Path(vault) / reference_path
        if verbose:
            print(f"\nIdentified reference '{reference}' in the vault '{vault}' as"
                f" the folder '{absolute_path}'...")
        reference_name = reference_path.name
    except NoteDoesNotExistError:
        reference_path = None
        absolute_path = None
        reference_name = path_name_no_ext(str(reference))
    
    template_note, reference_note = _find_template_and_reference_notes(vault, reference_name)
    delete = _confirm_for_deletion(confirm, reference_path, absolute_path,
                                   template_note, reference_note)

    if delete:
        _execute_deletion(reference_path, absolute_path, template_note,
                          reference_note, verbose)
    elif verbose:
        print(f"Aborting deleting reference.\n")


def _find_template_and_reference_notes(
        vault = PathLike,
        reference_name = str,
        ) -> tuple[VaultNote|None]: 
    """
    Helper function to `delete_reference_folder`.
    """
    template_note = VaultNote(vault, name=f'_template_{reference_name}')
    reference_note = VaultNote(vault, name=f'_reference_{reference_name}')
    if not template_note.rel_path_identified():
        template_note = None
    if not reference_note.rel_path_identified():
        reference_note = None
    return template_note, reference_note


def _confirm_for_deletion(
        confirm: bool,
        reference_path: PathLike,
        absolute_path: PathLike,
        template_note: VaultNote|None,
        reference_note: VaultNote|None,
        ) -> bool:
    """
    Helper function to `delete_reference_folder`.
    """
    if confirm:
        input_msg = _input_message(reference_path, absolute_path, template_note,
                                   reference_note)
        command = input(''.join(input_msg))
        delete = command == 'Y'
    else:
        delete = True
    return delete


def _input_message(
        reference_path: PathLike,
        absolute_path: PathLike,
        template_note: VaultNote|None,
        reference_note: VaultNote|None,
        ) -> list[str]:
    """
    Helper function to `delete_reference_folder`.
    """
    input_msg = [f"Delete"]
    if reference_path:
        input_msg.append(f"\n- all contents in the folder '{absolute_path}'")
    if template_note:
        input_msg.append(f"\n- '{template_note.path()}'")
    if reference_note:
        input_msg.append(f"\n- '{reference_note.path()}'")
    input_msg.append(f"?\n[Y/(n)]")
    return input_msg



def _execute_deletion(
        reference_path: PathLike,
        absolute_path: PathLike,
        template_note: VaultNote|None,
        reference_note: VaultNote|None,
        verbose: bool
        ) -> None:
    """
    Helper function to `delete_reference_folder`.
    """
    if verbose:
        print("Deleting...")
    if reference_path:
        shutil.rmtree(absolute_path)
        # TODO: delete the reference folder in a way that updates the cache.
    if template_note:
        template_note.delete()
    if reference_note:
        reference_note.delete()
    if verbose:
        print(f"Deleted reference.\n")




The `delete_reference_folder` method deletes the reference folder itself as well as the peripheral files for the reference (the template file and reference file) which are outside of the reference folder itself.

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.input', return_value='Y') as mock_input):
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)
    
    delete_reference_folder(temp_vault, reference='number_theory_reference_1')
    assert 'number_theory_reference_1' not in os.listdir(temp_vault / 'number_theory')
    assert not VaultNote(temp_vault, name='_template_number_theory_reference_1').exists()
    assert not VaultNote(temp_vault, name='_reference_number_theory_reference_1').exists()




Identified reference 'number_theory_reference_1' in the vault 'c:\Users\hyunj\Documents\Development\Python\trouver\nbs\temp_dirwpe9ozwq\test_vault_5' as the folder 'c:\Users\hyunj\Documents\Development\Python\trouver\nbs\temp_dirwpe9ozwq\test_vault_5\number_theory\number_theory_reference_1'...
Deleting...
Deleted reference.



In the following example, we test the deletion of a non-existent reference folder for which the template and reference files are deleted. 

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir,
      mock.patch('__main__.input', return_value='Y') as mock_input):
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)

    
    nonexistent_reference_name = 'nonexistent_reference'
    assert not VaultNote(temp_vault, name=f'_index_{nonexistent_reference_name}').exists()
    assert VaultNote(temp_vault, name=f'_template_{nonexistent_reference_name}').exists()
    assert VaultNote(temp_vault, name=f'_reference_{nonexistent_reference_name}').exists()

    delete_reference_folder(temp_vault, reference='nonexistent_reference')
    assert not VaultNote(temp_vault, name=f'_template_{nonexistent_reference_name}').exists()
    assert not VaultNote(temp_vault, name=f'_reference{nonexistent_reference_name}').exists()



Deleting...
Deleted reference.



## Helper methods for setting up new reference folders
Making a new reference folder takes some annoying amount of work.

In [ ]:
#| export
def _make_reference_folder(
        vault: Path, location: PathLike, reference_name: str,
        reference_directory: PathLike, overwrite: Union[str, None], verbose: bool) -> None:
    """Makes a folder of a specified name in the specified directory.
    
    If `overwrite` is `'w'`, then the reference folder is assumed to not
    exist.

    This method is Intended for making a reference folder for an Obsidian
    vault.
    
    **Parameters**
    - vault - PathLike
        - The path to the Obsidian vault in which to make the reference folder.
    - location - PathLike
        - The directory of the parent of the new folder to be made, relative
        to `vault`. 
    - reference_name - str
        - The name of the reference to be created; the folder's name will be
        this string.
    - reference_directory - PathLike
        - Is `location / reference_name`; in particular, this is a path
        relative to `vault`.
    - overwrite - `'w'`, `'a'`, or `None`.
        - Specifies if and how to overwrite the reference folder if it already
        exists.
            - If `'w'`, then the reference folder is assumed to not exist, and
            the reference folder is created.
            - If `'a'`, then the reference folder may or may not exist, and the
            reference folder is created if it does not exist.
            - If `None`, then the reference folder may or may not exist. If the
            reference folder exists, then a `FileExistsError` is raised. If the
            reference folder does not exist, then it is created.
    - verbose - bool
        - If `True`, print messages.
        
    **Raises**
    - FileNotFoundError
        - If `location` does not exist as a path relative to `vault`.
    - FileExistsError
        - If the reference folder already exists and 1. overwrite is `'w'` or
        2. overwrite is `'None'`. The former case is expected to not happen.
    """
    if verbose:
        print(f"Attempting to create a the folder '{reference_directory}'"
              f" in the directory '{vault / location}.'")

    if not os.path.exists(vault / location):
        raise FileNotFoundError(
            f"Attempted to create a reference folder at {vault / reference_directory}"
            f", but the parent directory {vault / location} does not" 
            " exist.")
    if os.path.exists(vault / reference_directory):
        if overwrite in ['w', None]:
            raise FileExistsError(
                "Attempted to create a reference folder at"
                f" {reference_directory}, but this directory already exists.")
    else:
        os.mkdir(vault / reference_directory)


In [ ]:
#| hide
test_vault = _test_directory() / 'test_vault_5'
with mock.patch('__main__.os.mkdir') as mock_os_mkdir:
    # Make a reference directory that does not yet exist with overwrite='w'.
    # The reference directory should be created with no errors.
    test_location_1 = Path('algebra')
    test_reference_name_1 = 'algebra_reference_1'
    test_reference_directory_1 = test_location_1 / test_reference_name_1
    _make_reference_folder(
        test_vault, location=test_location_1, reference_name=test_reference_name_1,
        reference_directory=test_reference_directory_1, overwrite='w', verbose=False)
    mock_os_mkdir.assert_called_with(test_vault / test_reference_directory_1)

    # Make a reference directory that does not yet exist with overwrite='a'.
    # The reference directory should be created with no errors.
    test_location_2 = Path('algebra')
    test_reference_name_2 = 'algebra_reference_2'
    test_reference_directory_2 = test_location_2 / test_reference_name_2
    _make_reference_folder(
        test_vault, location=test_location_2, reference_name=test_reference_name_2,
        reference_directory=test_reference_directory_2, overwrite='a', verbose=False)
    mock_os_mkdir.assert_called_with(test_vault / test_reference_directory_2)

    # Make a reference directory that does not yet exist with overwrite='None'.
    # The reference directory should be created with no errors.
    test_location_3 = Path('algebra')
    test_reference_name_3 = 'algebra_reference_3'
    test_reference_directory_3 = test_location_3 / test_reference_name_3
    _make_reference_folder(
        test_vault, location=test_location_3, reference_name=test_reference_name_3,
        reference_directory=test_reference_directory_3, overwrite=None, verbose=False)
    mock_os_mkdir.assert_called_with(test_vault / test_reference_directory_3)

    # Attempt to make a reference directory that exists with overwrite='w'.
    # The reference directory should not be created/modified and a FileExistsError should be
    # raised
    mock_os_mkdir.reset_mock()
    test_location_4 = Path('number_theory')
    test_reference_name_4 = 'number_theory_reference_1'
    test_reference_directory_4 = test_location_4 / test_reference_name_4
    with ExceptionExpected(FileExistsError):
        _make_reference_folder(
            test_vault, location=test_location_4, reference_name=test_reference_name_4,
            reference_directory=test_reference_directory_4, overwrite='w', verbose=False)
    mock_os_mkdir.assert_not_called()

    # Attempt to make a reference directory that exists with overwrite='a'.
    # The reference directory should not be modified and no errors should be
    # raised
    mock_os_mkdir.reset_mock()
    test_location_5 = Path('number_theory')
    test_reference_name_5 = 'number_theory_reference_1'
    test_reference_directory_5 = test_location_5 / test_reference_name_5
    _make_reference_folder(
        test_vault, location=test_location_5, reference_name=test_reference_name_5,
        reference_directory=test_reference_directory_5, overwrite='a', verbose=False)
    assert os.path.exists(test_vault / test_reference_directory_5)


    # Attempt to make a reference directory that exists with overwrite='None'.
    # The reference directory should not be created/modified and a FileExistsError should be
    # raised
    mock_os_mkdir.reset_mock()
    test_location_6 = Path('number_theory')
    test_reference_name_6 = 'number_theory_reference_1'
    test_reference_directory_6 = test_location_6 / test_reference_name_6
    with ExceptionExpected(FileExistsError):
        _make_reference_folder(
            test_vault, location=test_location_6, reference_name=test_reference_name_6,
            reference_directory=test_reference_directory_6, overwrite=None, verbose=False)
    mock_os_mkdir.assert_not_called()

In [ ]:
#| export
def _chapter_titles(chapters: list[Union[str, list[str]]]) -> list[str]:
    """
    Return the list of chapters of a reference from a formatted list of chapters
    and sections.

    **Parameters**
    - chapters - list[Union[str, list[str]]]
        - A list whose items are str or list of str. If an item is a string, then the
        item is the title of a chapter of the reference. If an item is a list of string,
        then the item contains the title of the chapter of the reference as its index-0
        item, and the titles of the sections for the chapter.

    **Returns**
     - list[str]
    """
    return [chapter if isinstance(chapter, str) else chapter[0] for chapter in chapters]

In [ ]:
#| hide
# Test _chapter_titles
chapters = [
    '0. Introduction',
    ['1. Some topic', 
     '1.1 blah',
     '1.2 blah']
]
test_eq(_chapter_titles(chapters), ['0. Introduction', '1. Some topic'])

In [ ]:
#| export
def _make_index_file(
        vault: PathLike, reference_directory: PathLike, reference_name: str,
        chapters: list[Union[str, list[str]]]) -> None:
    """
    Create the index file for the reference folder.

    The newly created index file will have a bulleted list of Obsidian links to the
    index files of the chapters/sections of the reference.

    **Parameters**
    - vault - PathLike
    - reference_directory - PathLike
    - reference_name - str
    - chapters - list[Union[str, list[str]]]
        - A list whose items are str or list of str. If an item is a string, then the
        item is the title of a chapter of the reference. If an item is a list of string,
        then the item contains the title of the chapter of the reference as its index-0
        item, and the titles of the sections for the chapter.
    """
    index_note = VaultNote(vault, rel_path = reference_directory / f'_index_{reference_name}.md')
    index_note.create()
    chapter_titles = _chapter_titles(chapters)

    chapter_bullets = [f'- [[_index_{convert_title_to_folder_name(chapter_title)}]]'
                        for chapter_title in chapter_titles]
    mf = MarkdownFile.from_list(chapter_bullets)
    mf.write(index_note)

In [ ]:
#| hide
test_vault = _test_directory() / 'test_vault_5'
test_reference_directory_1 = test_vault / 'mock_location' / 'mock_reference'
test_chapters = ['1. Chapter', '2. Another chapter', 'A. Appendix']
with (mock.patch('__main__.VaultNote.create') as mock_vaultnote_create,
      mock.patch('builtins.open', mock.mock_open()) as mock_open):
      # mock.patch('builtins.open', return_value=mock.MagicMock()) as mock_open,
      # mock.patch('sys.stdout.write') as mock_write):
    _make_index_file(test_vault, test_reference_directory_1, test_reference_name_1, test_chapters)
    mock_vaultnote_create.assert_called()
    mock_file = mock_open.return_value
    mock_write = mock_file.write

    mock_open.assert_called_once()
    mock_write.assert_called_once()
    args_passsed = mock_write.call_args.args
    single_arg_passed = args_passsed[0]
    test_eq(single_arg_passed.count('\n') + 1, len(test_chapters)) 
    assert '[[_index' in single_arg_passed

In [ ]:
#| export
def _make_chapter_folders_and_indices(
        chapters: list[Union[str, list[str]]],
        vault: PathLike,
        reference_directory: PathLike) -> None:
    chapter_titles = _chapter_titles(chapters)
    for chapter_title, chapter_sections in zip(chapter_titles, chapters):
        _make_single_chapter_folders(chapter_sections, chapter_title, vault, reference_directory)
        _make_single_chapter_index(chapter_sections, chapter_title, vault, reference_directory)

def _make_single_chapter_folders(
        chapter_sections: Union[str, list[str]],
        chapter_title: str,
        vault: Path,
        reference_directory: PathLike):
    chapter_title = convert_title_to_folder_name(chapter_title)
    os.mkdir(vault / reference_directory / chapter_title)

    if not isinstance(chapter_sections, list):  # i.e. chapter does not have sections
        return
    subchapter_titles = chapter_sections[1:]
    for subchapter_title in subchapter_titles:
        os.mkdir(vault / reference_directory / chapter_title /\
                    convert_title_to_folder_name(subchapter_title))


def _make_single_chapter_index(
        chapter_sections: Union[str, list[str]],
        chapter_title: str,
        vault: Path,
        reference_directory: PathLike):
    chapter_title = convert_title_to_folder_name(chapter_title)
    chapter_index_note = VaultNote(
        vault, rel_path=reference_directory / chapter_title\
        / f"_index_{chapter_title}.md")
    chapter_index_note.create()

    subchapter_titles = chapter_sections[1:]
    headings = [f'# {subchapter_title}\n\n\n'
                for subchapter_title in subchapter_titles]
    mf = MarkdownFile.from_list(headings)
    mf.write(chapter_index_note)

In [ ]:
#| hide
test_vault = _test_directory() / 'test_vault_5'
test_reference_directory_1 = Path('mock_location') / 'mock_reference'
test_chapters = ['1. Chapter', ['2. Another chapter', '2.1 Section', '2.2 Another Section'], ['A. Appendix', '1. Appendix section is formatted differently. It could happen.']]

with (mock.patch('__main__._make_single_chapter_folders') as mock_make_single_chapter_folders,
      mock.patch('__main__._make_single_chapter_index') as mock_make_single_chapter_index):
    _make_chapter_folders_and_indices(test_chapters, test_vault, test_reference_directory_1)
    # Assert that _make_single_chapter_folders and _make_single_chapter_index are each
    # called the appropriate number of times, i.e. 3 for this example.
    test_eq(mock_make_single_chapter_folders.call_count, len(test_chapters))
    test_eq(mock_make_single_chapter_index.call_count, len(test_chapters))

with mock.patch('__main__.os.mkdir') as mock_os_mkdir:
    # For '1. Chapter', only a single folder should be made
    _make_single_chapter_folders(
        chapter_sections=test_chapters[0], chapter_title='1. Chapter',
        vault=test_vault, reference_directory=test_reference_directory_1)
    test_eq(mock_os_mkdir.call_count, 1) 
    mock_os_mkdir.assert_called_with(test_vault / test_reference_directory_1 / convert_title_to_folder_name("1. Chapter"))

    # For '2. Chapter', 3 folders should be made - one for the chapter and two for sections in the chapter.
    mock_os_mkdir.reset_mock()
    _make_single_chapter_folders(
       chapter_sections=test_chapters[1], chapter_title='2. Chapter',
       vault=test_vault, reference_directory=test_reference_directory_1)
    test_eq(mock_os_mkdir.call_count, 3) 
    mock_os_mkdir.assert_any_call(test_vault / test_reference_directory_1 / convert_title_to_folder_name("2. Chapter"))
    mock_os_mkdir.assert_any_call(test_vault / test_reference_directory_1 / convert_title_to_folder_name('2. Chapter') / convert_title_to_folder_name(test_chapters[1][1]))
    mock_os_mkdir.assert_any_call(test_vault / test_reference_directory_1 / convert_title_to_folder_name('2. Chapter') / convert_title_to_folder_name(test_chapters[1][2]))

    # For '2. Chapter', 2 folders should be made - one for the chapter and one for the section in the chapter.
    mock_os_mkdir.reset_mock()
    _make_single_chapter_folders(
       chapter_sections=test_chapters[2], chapter_title='A. Appendix',
       vault=test_vault, reference_directory=test_reference_directory_1)
    test_eq(mock_os_mkdir.call_count, 2) 
    mock_os_mkdir.assert_any_call(test_vault / test_reference_directory_1 / convert_title_to_folder_name("A. Appendix"))
    mock_os_mkdir.assert_any_call(test_vault / test_reference_directory_1 / convert_title_to_folder_name('A. Appendix') / convert_title_to_folder_name(test_chapters[2][1]))


In [ ]:
#| export
def _make_reference_file(
        reference_name: str,
        references_folder: PathLike,
        vault: PathLike,
        reference_directory: PathLike, # Relative to `vault`. 
        create_reference_file_in_references_folder: bool,
        authors,
        author_files) -> None:
    """
    The references folder has subfolders 'A-E', 'F-J', etc. each of which contains
    subfolders for each letter in the English alphabet, each of which in turn contains
    reference notes
    """
    if create_reference_file_in_references_folder:
        if not os.path.exists(vault / references_folder):
            raise FileNotFoundError(
                f"References folder does not exist: {vault / references_folder}")

        if reference_name[0].isalpha():
            alphabet_group = alphabet_to_alphabet_group(reference_name[0])
            folder_to_make_reference_file = Path(references_folder)\
                / alphabet_group / reference_name[0].upper()
        else:
            folder_to_make_reference_file = Path(references_folder)
        reference_file_path = folder_to_make_reference_file\
            / f'_reference_{reference_name}.md'
    else:
        reference_file_path = reference_directory / f'_reference_{reference_name}.md'
    VaultNote(vault, rel_path=reference_file_path).create()
    # TODO author stuff

In [ ]:
#| hide
test_vault = _test_directory() / 'test_vault_5'
with mock.patch('__main__.VaultNote') as mock_VaultNote:
    _make_reference_file(
        'test_reference_1',
        Path('_references'),
        test_vault,
        None, # The `reference directory` parameter is not tested here.
        True, None, None)
    mock_VaultNote.assert_called_once_with(test_vault, rel_path=Path('_references') / 'P-T' / 'T' / '_reference_test_reference_1.md')
    mock_VaultNote.return_value.create.assert_called_once()

    mock_VaultNote.reset_mock()
    _make_reference_file(
        'kim_intro_num_theory',
        Path('_references'),
        test_vault,
        None, # The `reference directory` parameter is not tested here.
        True, None, None)
    mock_VaultNote.assert_called_once_with(test_vault, rel_path=Path('_references') / 'K-O' / 'K' / '_reference_kim_intro_num_theory.md')
    mock_VaultNote.return_value.create.assert_called_once()


In [ ]:
#| export


def _manifest_template_file(
        template_file_name: str,
        reference_name: str,
        vault: PathLike,
        authors: Union[str, list[str]])\
        -> MarkdownFile:
    """
    Return a `MarkdownFile` object based on a specified template file
    with some information about the reference added.

    **Parameters**
    - template_file_name: str
        - The template file's name. This is assumed to be unique in
        the vault.
    - reference_name: str
    - vault: PathLike
    - authors: Union[str, list[str]]

    **Returns**
    - MarkdownFile

    **Raises**
    - NoteNotUniqueError
        - If the template file's name is not unique in the vault.
    - NoteDoesNotExistError
        - If a note with the template file's name does not exist in the
        vault.
    """
    # TODO: sort authors by alphabetical order
    if isinstance(authors, str):
        authors = [authors]
    # TODO: get MarkdownFile by VaultNote
    # template_file_path = note_path_by_name(template_file_name, vault)
    # template_file = MarkdownFile.from_file(
    #     Path(vault) / template_file_path)
    try:
        template_file = MarkdownFile.from_vault_note(
            VaultNote(vault, name=template_file_name))
    except NotePathIsNotIdentifiedError:
        # TODO: test this possibility.
        template_file = MarkdownFile.from_string(
r'''---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note]
---
# Topic[^1]

# See Also

# Meta
## References

## Citations and Footnotes
[^1]: Citation
'''
        )
    embedding_link = ObsidianLink(is_embedded=True,
        file_name=f'_reference_{reference_name}',
        anchor=0, custom_text=0, link_type = LinkType.WIKILINK)
    template_file.add_line_in_section(
        title="References",
        line_dict={'type': MarkdownLineEnum.DEFAULT,
                'line': f'{embedding_link.to_string()}\n'})

    last_line = template_file.pop_line()
    last_line['line'] = f'[^1]: {", ".join(authors)}, '
    template_file.add_line_to_end(last_line)

    template_file.add_tags([f'#_reference/{reference_name}'])
    return template_file

In [ ]:
#| hide

# TODO: test
test_vault = _test_directory() / 'test_vault_5'
mf = _manifest_template_file('_template_common', reference_name='number_theory_reference_1', vault=test_vault, authors='Kim')
print(mf)
assert str(mf).endswith('Kim, ')

mf2 = _manifest_template_file('_template_common', reference_name='number_theory_reference_1', vault=test_vault, authors=['Kim', 'Eman'])
# print(mf2)
assert str(mf2).endswith('Kim, Eman, ')

---
cssclass: clean-embeds
aliases: []
tags: [_meta/literature_note, _reference/number_theory_reference_1]
---
# Topic[^1]

# See Also

# Meta
## References
![[_reference_number_theory_reference_1]]


## Citations and Footnotes
[^1]: Kim, 


In [ ]:
#| export
def _make_template_file(
        template_file_name,
        reference_name,
        vault,
        create_template_file_in_templates_folder: bool, 
        templates_folder,
        authors: Union[str, list[str]],
        make_second_template_file_in_reference_directory: bool,
        reference_directory: Path
        ) -> None:
    template_file = _manifest_template_file(
        template_file_name, reference_name, vault, authors)

    if create_template_file_in_templates_folder:
        # TODO factor out; this repeats with code in _make_reference_file above.
        if not os.path.exists(vault / templates_folder):
            raise FileNotFoundError(
                f"Templates folder does not exist: {vault / templates_folder}")
        if reference_name[0].isalpha():
            alphabet_group = alphabet_to_alphabet_group(reference_name[0])
            folder_to_make_template_file = Path(templates_folder)\
                / alphabet_group / reference_name[0].upper()
        else:
            folder_to_make_template_file = Path(templates_folder)
    else:
        folder_to_make_template_file = reference_directory
    template_file_path = folder_to_make_template_file\
        / f'_template_{reference_name}.md'
    _create_template_note_at(vault, template_file_path, template_file)

    if make_second_template_file_in_reference_directory:
        second_template_file_path = reference_directory / f'_template_{reference_name}_2.md'
        _create_template_note_at(vault, second_template_file_path, template_file)


def _create_template_note_at(
        vault: Path,
        note_path: Path, # Relative to `vault`
        template_file: MarkdownFile):
    template_note = VaultNote(vault, rel_path=note_path)
    template_note.create()
    template_file.write(template_note)

In [ ]:
#| hide
test_vault = _test_directory() / 'test_vault_5'
template_vault_note = VaultNote(test_vault, name='_template_common')
assert template_vault_note.exists()
with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__._manifest_template_file') as mock_manifest_template_file):

    _make_template_file(
        '_template_test_reference_1',
        'test_reference_1',
        test_vault,
        True,
        '_templates', ['Name1', 'Name2'],
        make_second_template_file_in_reference_directory=False, reference_directory=Path('test_reference_1'))
    mock_manifest_template_file.assert_called_once()
    mock_VaultNote.assert_called_once_with(test_vault, rel_path=Path('_templates') / 'P-T' / 'T' / '_template_test_reference_1.md')
    mock_VaultNote.return_value.create.assert_called_once()
    mock_manifest_template_file.return_value.write.assert_called_once()

    mock_VaultNote.reset_mock()
    mock_manifest_template_file.reset_mock()
    _make_template_file(
        '_template_kim_intro_num_theory',
        'kim_intro_num_theory',
        test_vault,
        True,
        '_templates', ['Kim'],
        make_second_template_file_in_reference_directory=False, reference_directory=Path('number_theory') / 'kim_intro_num_theory')
    mock_VaultNote.assert_called_once_with(test_vault, rel_path=Path('_templates') / 'K-O' / 'K' / '_template_kim_intro_num_theory.md')
    mock_VaultNote.return_value.create.assert_called_once()
    mock_manifest_template_file.return_value.write.assert_called_once()

    # Test macking template file as well as a second template file in the reference directory
    mock_VaultNote.reset_mock()
    mock_manifest_template_file.reset_mock()
    _make_template_file(
        '_template_test_reference_1',
        'test_reference_1',
        test_vault,
        True,
        '_templates', ['Name1', 'Name2'],
        make_second_template_file_in_reference_directory=True, reference_directory=Path('test_reference_1'))
    mock_manifest_template_file.assert_called_once()
    test_eq(mock_VaultNote.call_count, 2)
    mock_VaultNote.assert_any_call(test_vault, rel_path=Path('_templates') / 'P-T' / 'T' / '_template_test_reference_1.md')
    mock_VaultNote.assert_called_with(test_vault, rel_path=Path('test_reference_1') / '_template_test_reference_1_2.md')
    test_eq(mock_VaultNote.return_value.create.call_count, 2)
    test_eq(mock_manifest_template_file.return_value.write.call_count, 2)

In [ ]:
#| export
def _make_notation_index_file(
        reference_directory: PathLike,
        reference_name: str,
        vault: PathLike,
        notation_index_template_file_name: str,
        authors: list[str]) -> None:
    """Create the notation index file for the reference.

    The notation index file is named `_notation_{reference_name}.md`, and is
    located in the main directory of the new reference.

    **Parameters**
    - reference_directory - PathLike
        - The main directory of the reference. Relative to `vault`.
    - reference_name - str
    - vault - PathLike
    - notation_index_template_file_name - str
        - The template file whose contents will fill the newly created notation
        index file.
    
    **Raises**
    - NoteNotUniqueError
        - If the template file's name is not unique in the vault.
    - NoteDoesNotExistError
        - If a note with the template file's name does not exist in the
        vault.
    """
    template_file = _manifest_template_file(
        notation_index_template_file_name, reference_name, vault, authors)
    notation_index_file_path = Path(reference_directory)\
        / f'_notation_{reference_name}.md'
    notation_index_note = VaultNote(vault, rel_path=notation_index_file_path)
    notation_index_note.create()
    template_file.write(notation_index_note)

In [ ]:
#| hide
test_vault = _test_directory() / 'test_vault_5'
template_vault_note = VaultNote(test_vault, name='_template_notation_index')
assert template_vault_note.exists()
with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__._manifest_template_file') as mock_manifest_template_file):
    _make_notation_index_file(
        Path('mock_path_1') / 'test_reference_1', 'test_reference_1', test_vault, '_template_notation_index', ['Name1', 'Name2'])
    
    mock_manifest_template_file.assert_called_once()
    mock_VaultNote.assert_called_once_with(test_vault, rel_path=Path('mock_path_1') / 'test_reference_1' / '_notation_test_reference_1.md')
    mock_VaultNote.return_value.create.assert_called_once()

    # _make_notation_index_file('kim_intro_num_theory', Path('_references'), test_vault, None, None)
    # mock_VaultNote.assert_called_once_with(test_vault, rel_path=Path('_references') / 'K-O' / 'K' / '_reference_kim_intro_num_theory.md')
    # mock_VaultNote.return_value.create.assert_called_once()

In [ ]:
#| export
# TODO: change "file" in these helper methods and their respective doccstrings to "note"
def _make_glossary_file(
        reference_directory: PathLike, reference_name: str, vault: PathLike,
        glossary_template_file_name: str) -> None:
    """Create the glossary file for the reference.

    The notation index file is named `_glossary_{reference_name}.md`, and is
    located in the main directory of the new reference.

    **Parameters**
    - reference_directory - PathLike
        - The main directory of the reference.
    - reference_name - str
    - vault - PathLike
    - glossary_template_file_name - str
        - The template file whose contents will fill the newly created
        glossary file.
    
    """
    try:
        template_note = VaultNote(vault, name=glossary_template_file_name)
        template_file = MarkdownFile.from_vault_note(template_note)
        glossary_file_path = reference_directory / f'_glossary_{reference_name}.md'
        glossary_note = VaultNote(vault, rel_path=glossary_file_path)
        glossary_note.create()
        template_file.write(glossary_note)
    except NotePathIsNotIdentifiedError:
        return

In [ ]:
test_vault = _test_directory() / 'test_vault_5'
template_vault_note = VaultNote(test_vault, name='_template_glossary')
assert template_vault_note.exists()
with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__.MarkdownFile.from_vault_note') as mock_markdownfile_from_vault_note):
    _make_glossary_file(
        Path('mock_path_1') / 'test_reference_1', 'test_reference_1', test_vault, '_glossary_notation_index')
    
    test_eq(mock_VaultNote.call_count, 2)
    mock_VaultNote.assert_called_with(test_vault, rel_path=Path('mock_path_1') / 'test_reference_1' / '_glossary_test_reference_1.md')
    mock_VaultNote.return_value.create.assert_called_once()
    mock_markdownfile_from_vault_note.return_value.write.assert_called_once()

In [ ]:
#| export
def _make_temp_folder(
        reference_directory: PathLike, reference_name: str,
        vault: PathLike) -> None:
    """
    Make a folder for temporarily holding notes and a corresponding index file.

    **Parameters**
    - reference_directory - PathLike
    - reference_name - str
    - vault - PathLike

    """

    temp_directory = vault / reference_directory / '_temp'
    os.mkdir(temp_directory)
    # TODO: use vaultnote.create and markdownfile.write
    temp_file_path = reference_directory / '_temp' / f'_index_temp_{reference_name}.md'
    temp_note = VaultNote(vault, rel_path = temp_file_path)
    temp_note.create()

In [ ]:
test_vault = _test_directory() / 'test_vault_5'
with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__.os.mkdir') as mock_os_mkdir):
    _make_temp_folder(
        Path('mock_path_1') / 'test_reference_1', 'test_reference_1', test_vault)
    
    mock_os_mkdir.assert_called_once_with(test_vault / 'mock_path_1' / 'test_reference_1' / '_temp')
    mock_VaultNote.assert_called_once_with(test_vault, rel_path=Path('mock_path_1') / 'test_reference_1' / '_temp' / '_index_temp_test_reference_1.md')
    mock_VaultNote.return_value.create.assert_called_once()

## Copy obsidian vault plugin configs

When an Obsidian vault has a lot of notes (on the order of 10000+ notes), Obsidian tends to experience significant lag. As such, it can be useful to open a reference folder as a "subvault", i.e. a vault in itself, and in turn it is convenient to automatically copy settings from the "main" vault.

In [ ]:
#| export
def copy_obsidian_vault_configs(
        vault: PathLike,
        reference_directory: PathLike, # The folder into which to copy the Obsidian configs. Relative to `vault`.
        configs_folder: PathLike, # The folder containing the Obsidian configs. This is either an absolute path or relative to the current working directory.
        dirs_exist_ok: bool = False, # If `dirs_exist_ok` is `False` and the destination folder already exists, then a FileExistsError is raised. If `dirs_exist_ok` is true, the copying operation will continue if it encounters existing directories, and files within the destination tree will be overwritten by corresponding files from the source tree. See also the [`shutil.copytree`](https://docs.python.org/3/library/shutil.html#shutil.copytree) function.
        ignore = None, # See documentation of the [shutil.copytree](https://docs.python.org/3/library/shutil.html#shutil.copytree) function.
        ) -> None:
    """
    Copy the vault's Obsidian config files into the reference directory.

    `configs_folder` is copied into the destination `vault / reference_directory / '.obsidian'`.

    [`shutil.copytree`](https://docs.python.org/3/library/shutil.html#shutil.copytree) is used
    to copy the Obsidian configurations.

    **Raises**
    - `FileExistsError`
        - If `dirs_exist_ok` is `False` and the destination `vault / reference_directory / '.obsidian'`
          already exists.

    **See Also
    - `copy_obsidian_vault_configs_with_nice_modifications`
    """
    shutil.copytree(
        configs_folder, vault / reference_directory / '.obsidian',
        dirs_exist_ok=dirs_exist_ok, ignore=ignore)

In [ ]:
test_vault = _test_directory() / 'test_vault_5'
with (mock.patch('__main__.VaultNote') as mock_VaultNote,
      mock.patch('__main__.shutil.copytree') as mock_shutil_copytree):
    copy_obsidian_vault_configs(
        test_vault, Path('mock_path_1') / 'test_reference_1', test_vault / '.obsidian')
    
    mock_shutil_copytree.assert_called_once_with(
        test_vault / '.obsidian',
        test_vault / 'mock_path_1' / 'test_reference_1' / '.obsidian',
        dirs_exist_ok=False, ignore=None)

If `dirs_exist_okay = False` but then the `.obsidian` folder of `reference_directory` already exists, then a `FileExistsError` is raised, cf. the description of the `dirs_exist_ok` parameter of the [shutil.copytree](https://docs.python.org/3/library/shutil.html#shutil.copytree) function.

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    # First copy over a testing vault
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)

    # Copy the obsidian vault configs once
    copy_obsidian_vault_configs(
        temp_vault,
        Path('number_theory') / 'number_theory_reference_1',
        temp_vault / '.obsidian',
        dirs_exist_ok = False)
    
    # Then, try copying the configs again in the same way
    # into the same destination.
    with ExceptionExpected(ex=FileExistsError):
        copy_obsidian_vault_configs(
            temp_vault,
            Path('number_theory') / 'number_theory_reference_1',
            temp_vault / '.obsidian',
            dirs_exist_ok = False)

If `dirs_exist_okay = True`, and if the destination `vault / reference_directory / '.obsidian'` already exists, then the copying operation will continue if it encounters existing directories, and existing files will be overwritten  

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    # First copy over a testing vault
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)

    # Copy the obsidian vault configs once
    reference_directory = Path('number_theory') / 'number_theory_reference_1'
    copy_obsidian_vault_configs(
        temp_vault,
        reference_directory,
        temp_vault / '.obsidian',
        dirs_exist_ok = True)

    # Try modifying one of the files in the newly copied configs folder:
    with open(temp_vault / reference_directory / '.obsidian' / 'templates.json', 'w') as file:
        file.write('')
    
    # os.startfile(temp_vault / reference_directory)
    # input()

    # Then, try copying the configs again in the same way
    # into the same destination.
    copy_obsidian_vault_configs(
        temp_vault,
        reference_directory,
        temp_vault / '.obsidian',
        dirs_exist_ok = True)
    # The configs folder is copied over once again.
    with open(temp_vault / reference_directory / '.obsidian' / 'templates.json', 'r') as file:
        test_ne(file.read(), '')

We can use the `ignore` parameter to ignore copying over certain files (say satisfying specified patterns). See the `ignore` parameter of the [`shutil.copytree](https://docs.python.org/3/library/shutil.html#shutil.copytree) function.

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    # First copy over a testing vault
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)

    # Copy the obsidian vault configs once
    reference_directory = Path('number_theory') / 'number_theory_reference_1'
    copy_obsidian_vault_configs(
        temp_vault,
        reference_directory,
        temp_vault / '.obsidian',
        dirs_exist_ok = True)

    # Try modifying two of the files in the newly copied configs folder:
    with open(temp_vault / reference_directory / '.obsidian' / 'templates.json', 'w') as file:
        file.write('')
    with open(temp_vault / reference_directory / '.obsidian' / 'app.json', 'w') as file:
        file.write('')
    
    # os.startfile(temp_vault / reference_directory)
    # input()

    # Then, try copying the configs again in the same way
    # into the same destination, except for files named 'templates.json'.
    copy_obsidian_vault_configs(
        temp_vault,
        reference_directory,
        temp_vault / '.obsidian',
        dirs_exist_ok = True,
        ignore = shutil.ignore_patterns('templates.json'))
    # The configs folder is copied over once again, except for the 'templates.json' file.
    with open(temp_vault / reference_directory / '.obsidian' / 'templates.json', 'r') as file:
        test_eq(file.read(), '')
    with open(temp_vault / reference_directory / '.obsidian' / 'app.json', 'r') as file:
        test_ne(file.read(), '')

The author of `trouver` makes two configuration modifications for each reference/subvault  

1. In the author's own `fast-link-edit` plugin, the author sets the `referenceName` field to whatever the name of the reference is in the vault
2. In the `template` core plugin, the author sets the `folder` field to `.`.

Moreover, sometimes one might want to make modifications to the configurations in one's obsidian vaults; applying these modifications to all the configurations in the various subvaults would be tedious to do manually.

The below functions give ways to modify Obsidian plugin configurations and to copy and modify certain plugin configurations. 


In [ ]:
#| export
def _obsidian_vault_plugin_configs_file(
        vault: PathLike,
        plugin_name: str, # The folder name of the Obsidian plugin. This can be found either in the `.obsidian` directory or in the `.obsidian/plugins` directory in the vault .
        plugin_is_core: bool #`True` if the plugin is a core Obsidian.md plugin. `False` if the plugin is a community plugin.
        ) -> Path:
    if plugin_is_core:
        return Path(vault) / '.obsidian' / f'{plugin_name}.json'
    else:
        return Path(vault) / '.obsidian' / 'plugins' / plugin_name / 'data.json'

In [ ]:
#| hide
test_vault = _test_directory() / 'test_vault_5' 
configs_file_1 = _obsidian_vault_plugin_configs_file(test_vault, 'fast-link-edit', False)
assert os.path.exists(configs_file_1)

configs_file_2 = _obsidian_vault_plugin_configs_file(test_vault, 'templates', True)
assert os.path.exists(configs_file_2)

In [ ]:
#| export
def get_obsidian_vault_plugin_configs(
        vault: PathLike,
        plugin_name: str, # The folder name of the Obsidian plugin. This can be found in the `.obsidian` directory in the vault.
        plugin_is_core: bool #`True` if the plugin is a core Obsidian.md plugin. `False` if the plugin is a community plugin.
        ) -> dict[str, Union[str, int, float, bool, None, list, dict]]: # A json dict. 
    """Obtain the JSON object representing the `data.json` file of
    an Obsidian plugin.
    
    This function assumes that the the Obsidian plugin exists in `vault` in that the plugin
    has a `data.json` file.
    """
    with open(_obsidian_vault_plugin_configs_file(vault, plugin_name, plugin_is_core), 'r') as file:
        return json.load(file)
    

In [ ]:
test_vault = _test_directory() / 'test_vault_5' 
sample_output_1 = get_obsidian_vault_plugin_configs(test_vault, 'fast-link-edit', False)
test_eq(sample_output_1, {'referenceName': 'some_reference'})

sample_output_2 = get_obsidian_vault_plugin_configs(test_vault, 'templates', True)
test_eq(sample_output_2, {'folder': '_templates'})

In [ ]:
#| export
def modify_obsidian_vault_plugin_configs(
        vault: PathLike,
        plugin_name: str, # The folder name of the Obsidian plugin. This can be found in the `.obsidian` directory in the vault.
        plugin_is_core: bool, #`True` if the plugin is a core Obsidian.md plugin. `False` if the plugin is a community plugin.
        field: str, # The field to modify
        value: Union[str, int, float, bool, None, list, dict], # The JSON value to set the field as
        ) -> None:
    """Modify/set a top level field in an Obsidian vault plugin configs file.
    
    Assumes that the Obsidian vault plugins configs are indented by 2 spaces.

    Note that only top level values can be directly set by this function.

    """
    configs = get_obsidian_vault_plugin_configs(vault, plugin_name, plugin_is_core)
    configs[field] = value
    with open(_obsidian_vault_plugin_configs_file(vault, plugin_name, plugin_is_core), 'w') as file:
        json.dump(configs, file, indent=2)

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)

    modify_obsidian_vault_plugin_configs(temp_vault, 'fast-link-edit', False, 'referenceName', 'some_other_reference')
    sample_output_1 = get_obsidian_vault_plugin_configs(temp_vault, 'fast-link-edit', False)
    test_eq(sample_output_1, {'referenceName': 'some_other_reference'})

    modify_obsidian_vault_plugin_configs(temp_vault, 'templates', True, 'folder', '/')
    sample_output_2 = get_obsidian_vault_plugin_configs(temp_vault, 'templates', True)
    test_eq(sample_output_2, {'folder': '/'})


In [ ]:
#| export
def _modify_fast_link_edit_and_templates(
        vault: PathLike,
        reference_name: str, # The value to change the `referenceName` field in the `fast-link-edit` plugin into.
        template_location: str = '/' # The value to change the `folder` field in the `templates` plugin into.
    ) -> None:
    """Modify the `fast-link-edit` and `templates` Obsidian plugins for a vault if
    each exists."""
    try:
        modify_obsidian_vault_plugin_configs(
            vault, 'fast-link-edit', False, 'referenceName', reference_name)
    except FileNotFoundError:
        warnings.warn(
            "Attempted to modify the the new reference folder's configuration file"
            " for the `fast-link-edit` plugin, but the file does not exist.",
            UserWarning)

    try:
        modify_obsidian_vault_plugin_configs(
            vault, 'templates', True, 'folder', template_location)
    except FileNotFoundError:
        warnings.warn(
            "Attempted to modify the the new reference folder's configuration file"
            " for the `templates` plugin, but the file does not exist.",
            UserWarning)

In [ ]:
#| hide 

# Test that a warning occurs when the `fast-link-edit`` and `templates``
test_vault = _test_directory() / 'test_vault_5' 
# Note that `reference_directory` is being treated like a vault in itself.
reference_directory = test_vault / 'number_theory' / 'number_theory_reference_1'
assert os.path.exists(reference_directory)
assert not os.path.exists(reference_directory / '.obsidian')
test_warns(lambda: _modify_fast_link_edit_and_templates(reference_directory, 'number_theory_reference'))

# Test that `fast-link-edit` and `templates` are both modified if they both exist
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(test_vault, temp_vault)

    _modify_fast_link_edit_and_templates(temp_vault, 'just_some_reference_name', '_templates')
    configs_1 = get_obsidian_vault_plugin_configs(temp_vault, 'fast-link-edit', False)
    configs_2 = get_obsidian_vault_plugin_configs(temp_vault, 'templates', True)
    test_eq(configs_1['referenceName'], 'just_some_reference_name')
    test_eq(configs_2['folder'], '_templates')


# TODO: test that if one exists and the other doesn't then the one is modified  warning is raised and the other is not


In [ ]:
#| export
def copy_obsidian_vault_configs_with_nice_modifications(
        vault: PathLike,
        reference_directory: PathLike, # The folder into which to copy the Obsidian configs. Relative to `vault`.
        configs_folder: PathLike, # The folder containing the Obsidian configs to copy. This is either an absolute path or relative to the current working directory.
        reference_name: str = None, # The name of the reference and the value to change the `referenceName` field in the `fast-link-edit` plugin into. If `None`, then the reference name should be obtained as the name of `reference_directory`
        template_location: str = '/', # The location of the template file(s) and the value to change the `folder` field in the `templates` plugin into.
        dirs_exist_ok: bool = False, # If `dirs_exist_ok` is `False` and the destination folder already exists, then a FileExistsError is raised. If `dirs_exist_ok` is true, the copying operation will continue if it encounters existing directories, and files within the destination tree will be overwritten by corresponding files from the source tree. See also the [`shutil.copytree`](https://docs.python.org/3/library/shutil.html#shutil.copytree) function.
        ignore = None, # See documentation of the [shutil.copytree](https://docs.python.org/3/library/shutil.html#shutil.copytree) function.
        ) -> None:
    """
    Copy the vault's Obsidian config files into the reference directory and make some nice moodifications
    
    As with `copy_obsidian_vault_configs`,
    `configs_folder` is copied into the destination `vault / reference_directory / '.obsidian'`.

    **Raises**
    - FileExistsError
        - If `dirs_exist_ok` is `False` and the destination `vault / reference_directory / '.obsidian'`
          already exists.
    """
    copy_obsidian_vault_configs(
        vault, reference_directory, configs_folder, dirs_exist_ok=dirs_exist_ok,
        ignore=ignore)
    if reference_name is None:
        reference_name = path_name_no_ext(reference_directory)
    # Note that the config file that is being modified belongs to the "subvault",
    # i.e. the reference folder.
    _modify_fast_link_edit_and_templates(vault / reference_directory, reference_name, template_location)

In [ ]:
# TODO: test
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)

    # Copy the configs in the folder `.obsidian` at the root of temp_vault into the `number_theory_reference_1` folder
    copy_obsidian_vault_configs_with_nice_modifications(
        temp_vault, Path('number_theory') / 'number_theory_reference_1', temp_vault / '.obsidian')

    configs_1 = get_obsidian_vault_plugin_configs(temp_vault / 'number_theory' / 'number_theory_reference_1', 'fast-link-edit', False)
    configs_2 = get_obsidian_vault_plugin_configs(temp_vault / 'number_theory' / 'number_theory_reference_1', 'templates', True)
    test_eq(configs_1['referenceName'], 'number_theory_reference_1')
    test_eq(configs_2['folder'], '/')

    # TODO: test `dirs_exist_ok` being tested.

## Setting up a folder for a new reference


It takes a lot of time to setup a new reference in my Obsidian Math vault. I need to 

1. Make index notes
2. Make notation notes
3. Make mathematician notes for authors
4. Make template notes
5. Make subfolders
6. Type in titles/subsections in index notes
7. Make reference notes

In [ ]:
#| export
# TODO: add reference link in index note of parent folder
# TODO: Make another template file in the reference folder
# TODO: When copying over configs, change some things.
# TODO: add verbose option. 
#     Done: Verbose in _make_reference_folder
# TODO: add type hinting in hidden methods and docstrings.
# TODO: make a method for deleting reference.
# TODO: enable overwrite existing 
# TODO: make a function to detect connections between references.
# TODO: Implement overwrite existing files/overwrite entire folder option.
# TODO: for some reason, glossary isn't being added.
# TODO: test add obsidian config folder
# TODO: make TODO file


def setup_folder_for_new_reference(
        reference_name: str, # The name of the reference to be created; the folder's name will be this string.
        location: PathLike, # The directory of the parent of the new folder to be made, relative to `vault`.
        authors: Union[str, list[str]], # Each str is the family name of each author.
        vault: PathLike, # The path to the Obsidian vault in which to make the reference folder.
        author_folder: PathLike = '_mathematicians', # The directory where the author files are stored in.  Relative to `vault`. 
        create_reference_file_in_references_folder: bool = True, # If `True`, then the reference file creation is attempted within `references_folder`. Otherwise, the reference file creation is attempted at the base of the newly setup folder for the reference..
        references_folder: PathLike = '_references', # The directory where the references files are stored in.  Relative to `vault`.
        create_template_file_in_templates_folder: bool = True, # If `True`, then the template file creation is attempted within `templates_folder`. Otherwise, the template file creation is attempted at the base of the newly setup folder for the reference.
        templates_folder: PathLike = '_templates', # The directory where the template files are stored in.  Relative to `vault`.
        template_file_name: str = '_template_common', # The template file from which to base the template file of the new reference.
        notation_index_template_file_name: str = '_template_notation_index', # The template file from which to base the notation index file of the new reference.
        glossary_template_file_name: str = '_template_glossary', # The template file from which to base the glossary file of the new reference. Defaults to `'_template_glossary'`.
        chapters: Optional[list[Union[str, list[str]]]] = None, # A list where each item is either a str or a list of str.  If the item is a str, then it should be the title of a chapter named in the format `"1. {title}"`, `"2. {title}"`, or `"I. {title}"`, `"II. {title}"` etc. If the item is a list of str, then the 0th item should be the title of the chapter, formatted as in the previous sentence, and the other items should be titles of subchapters/subsections, also formatted in the same manner (e.g. if the subchapter is 7.2 of a book, then it should be `"2. {title}"`).  Defaults to `None`, in which case no chapters are specified and hence no chapter folders and indices are created.
        setup_temp_folder: bool = True, # If `True`, creates a `_temp` folder with an index file. This folder serves to house notes auto-created from LaTeX text files before moving them to their correct directories. Defaults to `True`.
        make_second_template_file_in_reference_directory: bool = True, # If `True`, creates a copy of the template note within the directory for the reference.
        copy_obsidian_configs: Optional[PathLike] = '.obsidian', # The folder relative to `vault` from which to copy obsidian configs.  If `None`, then no obsidian configs are copied to the reference folder. Defaults to `.obsidian`. 
        overwrite: Union[str, None] = None, # Specifies if and how to overwrite the reference folder if it already exists.  - If `'w'`, then deletes the contents of the existing reference folder, as well as the template and reference file before setting up the reference folder before creating the new reference folder.  - If `'a'`, then overwrites the contents of the reference folder, but does not remove existing files/folders.  - If `None`, then does not modify the existing reference folder and raises a `FileExistsError`.
        confirm_overwrite: bool = True, # Specifies whether or not to confirm the deletion of the reference folder if it already exists and if `overwrite` is `'w'`. Defaults to `True`.
        verbose: bool = False
        ) -> None:
    """Creates and sets up a new folder for a new reference.
    
    More specifically, the following are set up:
    
    1. The folder
    2. An index file
    3. Chapter folders and indices; the folder names are given by
      `convert_title_to_folder_name(<title_with_numbering>)` where
      `title_with_numbering` is the title of the chapter with its
      numbering if available, e.g. `1. Introduction`.
    4. Files in the `Mathematicians` folder, if applicable.
    5. A reference file
        - For embedding in the standard information notes for the reference
    6. A template file
    7. A glossary file
    8. A `#_meta/reference` tag for the new reference
    9. A notation index file
    10. Optionally, a `_temp` directory
    11. Optionally, the `./obsidian` config folder from the vault is
    copied.
    
    After they are setup, the user should
    1. Add content in the reference file.
    2. Modify the last line of the template file.
        
    **Raises**
    - FileExistsError
        - If `overwrite` is `None` and a folder of the name `reference_name`
        exists at `Path(vault) / location`.
    """
    if verbose:
        print("Setting up a new folder for a reference.")
        print(f"\tReference name: {reference_name}")
        print(f"\tVault: {vault}")
        print(f"\tLocation of new folder (relative to vault): {location}")
    vault = Path(vault)
    if not chapters:
        chapters = []
    reference_directory = Path(location) / reference_name

    if overwrite == 'w' and os.path.exists(vault / reference_directory):
        delete_reference_folder(
            vault, reference_directory, verbose=verbose,
            confirm=confirm_overwrite)

    _make_reference_folder(
        vault, location, reference_name, reference_directory,
        overwrite=overwrite, verbose=verbose)
    _make_index_file(vault, reference_directory, reference_name, chapters)
    _make_chapter_folders_and_indices(chapters, vault, reference_directory)

    # Look for/make mathematician files

    # TODO: `author_files` ultimately does not really do anything. Change this
    author_files = []
    # for author in authors:
    #     author_file = find_author_file(vault, author, author_folder)
    #     if author_file is None:
    #         print('')  # TODO
    #     author_files.append(author_file)

    _make_reference_file(
        reference_name, references_folder, vault,
        reference_directory, create_reference_file_in_references_folder,
        authors, author_files)
    _make_template_file(
        template_file_name, reference_name, vault,
        create_template_file_in_templates_folder,
        templates_folder, authors, make_second_template_file_in_reference_directory,
        reference_directory)
    _make_notation_index_file(
        reference_directory, reference_name, vault,
        notation_index_template_file_name, authors)
    _make_glossary_file(
        reference_directory, reference_name, vault,
        glossary_template_file_name)
    if setup_temp_folder:
        _make_temp_folder(reference_directory, reference_name, vault)
    if copy_obsidian_configs is not None:
        configs_folder = vault / copy_obsidian_configs
        # copy_obsidian_vault_configs(vault, reference_directory, configs_folder)
        copy_obsidian_vault_configs_with_nice_modifications(
            vault, reference_directory, configs_folder)
        # Modify 
    if verbose:
        print(f'Created a new reference folder at {location}.'\
            ' Make sure to update the reference file and'\
            ' The files for new mathematicians!')

We can set up a nicely formatted reference folder (without content) with the `setup_folder_for_new_reference` method. Try uncommenting the `os.startfile` and `os.input` lines to see what the vault looks like once the reference is setup.

In [ ]:
with (tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir):
    temp_vault = Path(temp_dir) / 'test_vault_5'
    shutil.copytree(_test_directory() / 'test_vault_5', temp_vault)

    chapters= [  # Chapters/sections and sections/subsections of the reference can be set up
        '0. Having a chapter 0 is fine',
        ['1. Introduction', '1.1. A subsection'],
        ['2. Basic ring theory', '2.1. Rings', '2.2. Integral domains', '2.3. Fields', '2.4 Fermat\'s Little Theorem'],
        '3. Chapter without subsection',
        ['A. Appendix: Set theory', 'A.1 Set Axioms', 'A.2 Constructions']
    ]
    setup_folder_for_new_reference(
        reference_name='kim_intro_num_theory', location='number_theory',
        authors='Kim', vault=temp_vault, chapters=chapters
        )

    # os.startfile(temp_vault)
    # input()
    # TODO: test that files that need to be created have been created.

TypeError: _make_template_file() takes 7 positional arguments but 9 were given

In [ ]:
# TODO: test overwrite and confirm_overwrite parameters
# TODO: give an example of verbose 

## Getting the chapters/sections from a file

## Getting subfolders and index files of a reference

In [ ]:
#| export
def get_index_notes_from_subdirectory(
        vault: PathLike, # The path to the Obsidian vault directory.
        subdirectory: PathLike, # The path, relative to `vault` of the subdirectory of of the vault for the reference.
        main_index_note: bool = False, # If `True`, include the main index note for the reference. This index note should be in the directory specified by `subdirectory`.
        as_vault_notes: bool = True # If `True`, returns the ``VaultNote`` objects for the index notes.  Otherwise, returns the paths to the index notes as paths, represented by str.
        ) -> list[Union[str, VaultNote]]: # Either of the names of the index notes in the vault or of the index notes as VaultNote objects, depending on `as_vault_notes`.
    """Returns list of index note files for reference
    """
    vault = Path(vault)
    reference_directory = vault / subdirectory
    index_notes = glob.glob(f'{reference_directory}/**/_index_*.md', recursive=True)
    index_notes = [index_note for index_note in index_notes 
                   if Path(os.path.dirname(index_note)) != reference_directory / "_temp"]
    if not main_index_note:
        index_notes = [index_note for index_note in index_notes
                       if Path(os.path.dirname(index_note)) != reference_directory]
    index_notes = [os.path.relpath(index_note, vault) for index_note in index_notes]
    if as_vault_notes:
        index_notes = [VaultNote(vault, rel_path=index_note) for index_note in index_notes]
    return index_notes

In [ ]:
# TODO: test

In [ ]:
#| export
# TODO: reformat
def get_index_notes_from_index_note(
        vault: PathLike, reference_name: str,
        as_vault_notes: bool = True) -> list[Union[str, VaultNote]]:
    """Returns the list of index notes for the reference in the order
    that they are listed in the reference's main index note.
    
    Does so by searching the "main" index note in the reference folder.
    Returns the index notes in order that they are listed in the main index
    note.
    
    Assumes that the reference folder is "formatted correctly". This includes
    the assumption that the main index note for the reference is the unique
    note named `'_index_{reference_name}'` in the vault.
    
    **Parameters**

    - `vault` - PathLike
        - The path to the Obsidian vault directory
    - `reference_name` - str
        - The name of the reference folder in the vault.
    - `as_vault_notes` - bool
        - If `True`, returns the ``VaultNote`` objects for the index notes.
        Otherwise, returns the paths to the index notes as paths, represented
        by str. Defaults to `True`.

    **Returns**
    - list[Union[str, VaultNote]]
        - Either of the names of the index notes in the vault or of the 
        index notes as VaultNote objects, depending on `as_vault_notes`.
    
    **See Also**
    - ``get_notes_from_index_note`` in
    ``markdown.obsidian.personal.index_notes``.
    """
    vault = Path(vault)
    main_index_note = VaultNote(vault, name=f'_index_{reference_name}')
    mf = MarkdownFile.from_vault_note(main_index_note)
    text = str(mf)
    links = links_from_text(text)
    index_notes = [link.file_name for link in links]
    if as_vault_notes:
        index_notes = [VaultNote(vault, name=index_note)
                       for index_note in index_notes]
    return index_notes

In [ ]:
# TODO: test

## Finding all reference folders in a vault

In [ ]:
#| export
def reference_folders_in_vault(vault: PathLike) -> dict[str, str]:
    """Returns a dict of reference folders in vault.
    
    **Parameters**
    - vault - PathLike
        - The path to the Obsidian vault directory.

    **Returns**
    - dict
        - The keys are str, each of which is the name of a reference.
        The values are path strings to the reference folder, relative to
        `vault`.
    """
    vault = Path(MATH_VAULT_LOCATION)
    reference_folders = {}
    index_files = vault.glob(f'**/_index*.md')
    for index_file in index_files:
        # print(index_file)
        directory = os.path.dirname(index_file)
        # print(directory)
        directory = os.path.relpath(directory, vault)
        # print(directory)
        # print(f'{directory}/_notation*.md')
        notation_notes = vault.glob(f'{directory}/_notation*.md')
        if list(notation_notes):
            reference_name = os.path.basename(directory)
            reference_folders[reference_name] = directory
            if not os.path.exists(vault / directory /\
                                  f'_index_{reference_name}.md'):
                warnings.warn(f'''The index file for this reference
                              seems misnamed: {reference_name}''')
            if not os.path.exists(vault / directory /\
                                 f'_notation_{reference_name}.md'):
                warnings.warn(f'''The notation file for this reference
                              seems misnamed: {reference_name}''')
    return reference_folders


In [ ]:
# TODO: example

## Getting files in a reference folder

In [ ]:
#| export
# TODO: reformat
def files_in_reference_folder(
        vault: PathLike, reference: str, as_list: bool = False,
        search_index_notes: bool = False,
        exclude_notes_of_type: list[PersonalNoteTypeEnum] = None)\
        -> Union[list[str], dict[str, str]]:
    """Returns a dict or list of files in a reference folder of a vault.

    TODO: also implement a method to get the files as listed in index notes.

    **Parameters**
    - vault - PathLike
    - reference - str
        - Name of the reference folder in the vault. The folder must have
        the index note file `'_index_{reference}.md'`, and there must not
        be two index note files of the same name in the vault.
    - as_list - bool
        - If `True`, then returns a list. Returns a dict otherwise. Defaults
        to `False`.
    - search_index_notes - bool
        - If `True`, then gets the notes by looking at the reference's index
        notes and returns the notes as ordered by the index notes.
    - exclude_notes_of_type - list of 
            ``markdown.obsidian.personal.note_type.PersonalNoteTypeEnum``.
        - The notes of the types specified here will be excluded in the dict
        or list returned. Defaults to `None`, in which case
        
    **Returns**
    - list[str] or dict[str, str]
        - Either a list of path strings or a dict whose keys are str and values
        are path strings. The keys are the names of the files. The Paths are
        relative to `vault`.
    """
    if not exclude_notes_of_type:
        exclude_notes_of_type = []
    vault = Path(vault)
    if search_index_notes:
        return _files_in_reference_folder_via_index_notes(
            vault, reference, as_list, exclude_notes_of_type)
    else:
        return _files_in_reference_folder_via_glob(
            vault, reference, as_list, exclude_notes_of_type)
    
def _files_in_reference_folder_via_index_notes(
        vault: PathLike, reference: str, as_list: bool,
        exclude_notes_of_type: list[PersonalNoteTypeEnum])\
        -> Union[list[str], dict[str, str]]:
    # TODO make sure that notes_linked_in_note is ordered correctly.
    index_of_index = VaultNote(vault, name=f'_index_{reference}')
    index_notes = notes_linked_in_note(index_of_index, as_dict=False)
    file_list = []
    for index_note in index_notes:
        notes_in_index_note = notes_linked_in_note(index_note, as_dict=False)
        file_list.extend(notes_in_index_note)
    file_list = [note.rel_path for note in file_list
                 if type_of_note(note) not in exclude_notes_of_type]
    if as_list:
        return file_list
    else:
        return {os.path.basename(file): file for file in file_list}
    
def _files_in_reference_folder_via_glob(
        vault: PathLike, reference: str, as_list: bool,
        exclude_notes_of_type: list[PersonalNoteTypeEnum])\
        -> Union[list[str], dict[str, str]]:
    vn = VaultNote(vault, name=f'_index_{reference}')
    reference_folder = Path(os.path.dirname(vn.path()))
    files = reference_folder.glob('**/*.md')
    file_list = [os.path.relpath(Path(file), start=vault) for file in files] 
    vault_notes = [VaultNote(vault, rel_path=file) for file in file_list]
    vault_notes = [vn for vn in vault_notes
                 if type_of_note(vn) not in exclude_notes_of_type]
    file_list = [vn.rel_path for vn in vault_notes]
    if as_list:
        return file_list
    else:
        return {os.path.basename(file): file for file in file_list}

In [ ]:
# TODO: examples